In [1]:
#Import Packages
from selenium import webdriver
import time
import pandas as pd
import os
#Import Packages

from datetime import datetime
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [2]:
# !pip install selenium
# !pip install pandas

In [3]:
url1='https://www.linkedin.com/jobs/search/?keywords=Data%20Analyst&location=United%20States&locationId=&geoId=103644278&f_TPR=r604800&f_E=2%2C3&f_PP=104194190&position=1&pageNum=0'

# service = Service('C:\\Users\\suraj\\OneDrive - UNT System\\2023\\softwares\\chromedriver_win32\\chromedriver.exe')
options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(service=service, options=options)
driver = webdriver.Chrome(options = options)
driver.maximize_window()
driver.implicitly_wait(3)
driver.get(url1)

In [4]:
y=driver.find_elements(By.CLASS_NAME,'results-context-header__job-count')[0].text
y=y.strip('+')
y=y.replace(",", "")
y=pd.to_numeric(y)
y

264

In [5]:
i=1
# scroll_position = 1  # Example value: Adjust this as needed
time.sleep(4)
# for k in range(1,int(y/10)):
scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;") 
    try:
        send=driver.find_element(By.XPATH,"//button[@aria-label='See more jobs']")
        driver.execute_script("arguments[0].click();", send)   
        time.sleep(3)
    except:
        pass
        time.sleep(1)
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     time.sleep(2)
# #     scroll_height = driver.execute_script("return document.body.scrollHeight")

#     try:
#         send=driver.find_element(By.XPATH,"//button[@aria-label='See more jobs']")
#         driver.execute_script("arguments[0].click();", send)   
#         time.sleep(3)
#     except:
#         pass
#         time.sleep(3)
        
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight-10);") 
#     time.sleep(2)
       
    
#     element = driver.find_element("","").send_keys(Keys.PAGE_DOWN)
#     element.send_keys("hey")
#     driver.send_keys(Keys.PAGE_DOWN)
    

In [6]:
#Create empty lists for company name and job title

companyname= []


#Find company name and append it to the blank list

for j in range(y):
    try:
        company=driver.find_elements(By.CLASS_NAME,'base-search-card__subtitle')[j].text
        companyname.append(company)
#         title=driver.find_elements(By.CLASS_NAME,'base-search-card__title')[j].text
#         titlename.append(title) 
    except :
        pass
#         print("no")
#         print(company)
  

In [7]:
#Create empty lists for company name and job title

titlename= []


#Find company name and append it to the blank list

for i in range(y):
    try:
#         company=driver.find_elements(By.CLASS_NAME,'base-search-card__subtitle')[i].text
#         companyname.append(company)
        title=driver.find_elements(By.CLASS_NAME,'base-search-card__title')[i].text
        titlename.append(title) 
    except :
        pass
#         print("no")
#         print(company)
  

In [8]:
len(companyname)

264

In [9]:
# companyname

In [10]:
len(titlename)

264

In [11]:
companyname=pd.DataFrame(companyname,columns=["Company"])
titlename=pd.DataFrame(titlename,columns=['Title'])
# companyname

In [12]:
# combined_df = companyname.join(titlename)
# combined_df

combined_df=pd.concat([companyname,titlename],axis=1)
combined_df

,Company,Title
0,Anblicks,Data Analyst
1,Central Business Solutions Inc.,Data Analyst
2,Get It Recruit - Information Technology,Data Analyst Salesforce - Remote | WFH
3,Anblicks,Data Analyst role
4,Mindpal,Data Engineer (Snowflake)
...,...,...
259,Dice,DevsecOps Engineer
260,STV,SIT Survey Technician - 4416
261,Dice,W2 Opportunity - iOS Developer
262,Dice,Java Lead


In [13]:
jobList=driver.find_elements(By.CLASS_NAME,"base-card__full-link")
hrefList = []
for e in jobList:
    hrefList.append(e.get_attribute('href'))

# hrefList = []
# for e in link_lists:
#     hrefList.append(e.get_attribute('href'))

# for link in driver.findAll('a'):
#     print(link.get('href'))

In [14]:
# hrefList
# jobposted=driver.find_elements(By.CLASS_NAME,"job-search-card__listdate")
# posted=[]
# for d in jobposted:
#     posted.append(driver.find_element(By.TAG_NAME, 'datetime'))
    
# posted

time_elements = driver.find_elements(By.CLASS_NAME, 'time')
time_texts = [element.text for element in time_elements]

# Print the extracted time texts
for time_text in time_texts:
    print("Time:", time_text)
time_texts

[]

In [15]:
#Create empty lists for company name and job title

last_updated= []
postdate=[]


#Find company name and append it to the blank list

for d in range(y):
    try:
        posting=driver.find_elements(By.CLASS_NAME,'job-search-card__listdate')[d].text
        datetime_value = driver.find_elements(By.CLASS_NAME,'job-search-card__listdate')[d].get_attribute('datetime')
        postdate.append(datetime_value)
        last_updated.append(posting)
#         title=driver.find_elements(By.CLASS_NAME,'base-search-card__title')[j].text
#         titlename.append(title) 
    except :
        pass
#         print("no")
#         print(company)

In [16]:
postdate=pd.DataFrame(postdate,columns=["postdate"])
last_updated=pd.DataFrame(last_updated,columns=["last_updated"])

In [17]:
dt=datetime.now().strftime("%Y-%m-%d %H-%M-%S")
print(dt)

2023-07-07 16-51-00


In [18]:
linklist=pd.DataFrame(hrefList,columns=["joblinks"])
# combined_df = linklist.join(combined_df)
combined_df = pd.concat([postdate,combined_df,linklist,last_updated], axis=1)
combined_df
filename= 'linkedIn_jobs'+ dt+'.csv'
combined_df.to_csv(filename,index=False)

#Close the driver

driver.close()